### This is a *brief* summary of what the assignment consisted of:

#### Question 1: Generate synthetic data sets, 2 classed with 20 features each, classes share Covariance matrix
#### Question 2: Estimate parameters of the GDA model using maximul likelihood
#### Question 3: Use k-NN to learn a classifier
#### Question 4: Generate synthetic data sets, each class now a mixture of 3 Gaussians
#### Question 5: Same as Q2 and Q3 with new data set

In [1]:
import math
import numpy as np
from numpy.linalg import inv
from numpy.linalg import norm
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import operator

In [2]:
class Data:
    
    def __init__(self):
        self.m0 = []
        self.m1 = []
        self.m2 = []
        self.c1m1 = []
        self.c1m2 = []
        self.c1m3 = []
        self.c2m1 = []
        self.c2m2 = []
        self.c2m3 = []
        self.cov1 = []
        self.cov2 = []
        self.cov3 = []
        
    def get_m(self, filename):
        f = open(filename)
        line = f.readline().split(",")
        line = line[:20]
        m = []
        for x in line:
            m.append(float(x))
        return m
    
    def get_cov(self, filename):
        f = open(filename)
        lines = []
        for line in f.readlines():
            line = line.strip().split(",")
            line = line[:20]
            for idx, x in enumerate(line):
                line[idx] = float(x)
            lines.append(line)
        lines = np.array(lines)
        return lines

In [3]:
ds1 = Data()
ds2 = Data()
ds1.m0 = ds1.get_m('Datasets/DS1_m_0.txt')
ds1.m1 = ds1.get_m('Datasets/DS1_m_1.txt')
ds2.c0m1 = ds2.get_m('Datasets/DS2_c1_m1.txt')
ds2.c0m2 = ds2.get_m('Datasets/DS2_c1_m2.txt')
ds2.c0m3 = ds2.get_m('Datasets/DS2_c1_m3.txt')
ds2.c1m1 = ds2.get_m('Datasets/DS2_c2_m1.txt')
ds2.c1m2 = ds2.get_m('Datasets/DS2_c2_m2.txt')
ds2.c1m3 = ds2.get_m('Datasets/DS2_c2_m3.txt')
ds1.cov = ds1.get_cov('Datasets/DS1_Cov.txt')
ds2.cov1 = ds2.get_cov('Datasets/DS2_Cov1.txt')
ds2.cov2 = ds2.get_cov('Datasets/DS2_Cov2.txt')
ds2.cov3 = ds2.get_cov('Datasets/DS2_Cov3.txt')

## Q1/Q4:

In [4]:
def DS(c):
    np.random.seed(0)
    if(c == 1):
        neg_m = np.random.multivariate_normal(ds1.m0, ds1.cov, 2000)
        neg = np.zeros((2000,21))
        neg[:,:-1] = neg_m
        pos_m = np.random.multivariate_normal(ds1.m1, ds1.cov, 2000)
        pos = np.ones((2000,21))
        pos[:,:-1] = pos_m
        return neg, pos
    if(c == 2):
        neg_m = []
        for i in range(2000):
            distr = np.random.choice([0,1,2], p=(0.1,0.42,0.48))
            if(distr == 0):
                neg_m.append(np.random.multivariate_normal(ds2.c0m1, ds2.cov1))
            if(distr == 1):
                neg_m.append(np.random.multivariate_normal(ds2.c0m2, ds2.cov2))
            if(distr == 2):
                neg_m.append(np.random.multivariate_normal(ds2.c0m3, ds2.cov3))
        neg = np.zeros((2000,21))
        neg[:,:-1] = neg_m
        pos_m = []
        for i in range(2000):
            distr = np.random.choice([0,1,2], p=(0.1,0.42,0.48))
            if(distr == 0):
                pos_m.append(np.random.multivariate_normal(ds2.c1m1, ds2.cov1))
            if(distr == 1):
                pos_m.append(np.random.multivariate_normal(ds2.c1m2, ds2.cov2))
            if(distr == 2):
                pos_m.append(np.random.multivariate_normal(ds2.c1m3, ds2.cov3))
        pos = np.ones((2000,21))
        pos[:,:-1] = pos_m
        return neg, pos

## Adjust first line of code below and re-run cells afterwards: 
## Q2/Q3: DS(1)  
## Q5: DS(2)

In [18]:
neg, pos = DS(2) # choose 1 for Q2 and Q3, choose 2 for Q5
pos_train, pos_test, neg_train, neg_test = train_test_split(pos, neg, test_size=0.2, random_state=1)
pos_train, pos_val, neg_train, neg_val = train_test_split(pos_train, neg_train, test_size=0.25, random_state=1)
train = np.append(pos_train, neg_train, axis=0)
np.random.shuffle(train)
val = np.append(pos_val, neg_val, axis=0)
np.random.shuffle(val)
test = np.append(pos_test, neg_test, axis=0)
np.random.shuffle(test)
#np.savetxt("train2.csv", train, delimiter=',', fmt='%1.3f')
#np.savetxt("validation2.csv", val, delimiter=',', fmt='%1.3f')
#np.savetxt("test2.csv", test, delimiter=',', fmt='%1.3f')

In [19]:
m0 = np.zeros(20)
m1 = np.zeros(20)
ct0 = 0
ct1 = 0
for x in train:
    if(int(x[20]) == 0):
        m0 = np.add(m0, x[0:20])
        ct0 += 1
    elif(int(x[20]) == 1):
        m1 = np.add(m1, x[0:20]) 
        ct1 += 1
m0 /= ct0
m1 /= ct1

In [20]:
s0 = np.zeros((20,20))
s1 = np.zeros((20,20))
ct0 = 0
ct1 = 0
for x in train:
    if(int(x[20]) == 0):
        s0 = np.add(s0, np.outer(np.subtract(x[0:20], m0), np.subtract(x[0:20], m0)))
        ct0 += 1
    elif(int(x[20]) == 1):
        s1 = np.add(s1, np.outer(np.subtract(x[0:20], m1), np.subtract(x[0:20], m1)))
        ct1 += 1
s0 /= ct0
s1 /= ct1
s = ct0/len(train)*s0 + ct1/len(train)*s1

In [21]:
w = np.matmul(inv(s), np.subtract(m0, m1))
w0 = -0.5*np.matmul(np.transpose(m0), np.matmul(inv(s), m0)) + 0.5*np.matmul(np.transpose(m1), np.matmul(inv(s), m1))
print(w, w0)

[ 0.01042424 -0.01689383  0.10482438 -0.067629   -0.09361109  0.02332355
 -0.02329479  0.07333374 -0.01225595 -0.05225086 -0.08276808 -0.03543034
  0.00705043 -0.02072892  0.07734468  0.10205874  0.01008776 -0.04366477
  0.00151375 -0.03768648] 0.087027177878134


In [22]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [23]:
y = 0.
correct = 0
conf = np.zeros((2,2))
for x in test:
    y = 1 - np.round(sigmoid(np.inner(np.transpose(w), x[0:20]) + w0))
    if(y == x[20]):
        correct += 1
    if(y == 0 and x[20] == 0):
        conf[1][1] += 1
    if(y == 1 and x[20] == 1):
        conf[0][0] += 1
    if(y == 0 and x[20] == 1):
        conf[1][0] += 1
    if(y == 1 and x[20] == 0):
        conf[0][1] += 1
accuracy = correct / len(test)
precision = conf[0][0]/(conf[0][0] + conf[0][1])
recall = conf[0][0]/(conf[0][0] + conf[1][0])
        
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", 2*precision*recall / (precision+recall))

Accuracy:  0.50125
Precision: 0.5012468827930174
Recall: 0.5025
F-measure: 0.50187265917603


In [24]:
def classify(train, testInst, k):
    count0 = 0
    distances = []
    for x in range(len(train)):
        dist = norm(train[x]-testInst)
        if(dist != 0.0):
            distances.append((train[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        if(distances[x][0][20] == 0.0):
            count0 += 1
        neighbors.append(distances[x][0])
    return np.round(1-count0/k)

In [25]:
def get_F(k, pr):
    correct = 0
    conf = np.zeros((2,2))
    for x in train:
        y = classify(train, x, k)
        if(y == x[20]):
            correct += 1
        if(y == 0 and x[20] == 0):
            conf[1][1] += 1
        if(y == 1 and x[20] == 1):
            conf[0][0] += 1
        if(y == 0 and x[20] == 1):
            conf[1][0] += 1
        if(y == 1 and x[20] == 0):
            conf[0][1] += 1
    accuracy = correct / len(train)
    precision = conf[0][0]/(conf[0][0] + conf[0][1])
    recall = conf[0][0]/(conf[0][0] + conf[1][0])
    f_measure = 2*precision*recall / (precision+recall)
    print("k:", k, "F-measure:", f_measure)
    if(pr == 1):
        print("Accuracy: ", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)

## The next few cells would take a while to run, hence the commenting.

In [26]:
#for k in range(1, 102, 10):
#    get_F(k, 0)

In [27]:
#for k in range(111, 132, 10):
#   get_F(k, 0)

In [28]:
#get_F(151, 0)

In [29]:
# for k in range(51, 122, 10): # range(11, 82, 10) for DS(2)
#     correct = 0
#     conf = np.zeros((2,2))
#     for x in val:
#         y = classify(train, x, k)
#         if(y == x[20]):
#             correct += 1
#         if(y == 0 and x[20] == 0):
#             conf[1][1] += 1
#         if(y == 1 and x[20] == 1):
#             conf[0][0] += 1
#         if(y == 0 and x[20] == 1):
#             conf[1][0] += 1
#         if(y == 1 and x[20] == 0):
#             conf[0][1] += 1
#     accuracy = correct / len(val)
#     precision = conf[0][0]/(conf[0][0] + conf[0][1])
#     recall = conf[0][0]/(conf[0][0] + conf[1][0])
#     f_measure = 2*precision*recall / (precision+recall)
#     print("k:", k, "F-measure:", f_measure)
#     print("Accuracy: ", accuracy)
#     print("Precision:", precision)
#     print("Recall:", recall) 

In [ ]:
k = 31  # k = 31 for DS(2)
correct = 0
conf = np.zeros((2,2))
for x in test:
    y = classify(train, x, k)
    if(y == x[20]):
        correct += 1
    if(y == 0 and x[20] == 0):
        conf[1][1] += 1
    if(y == 1 and x[20] == 1):
        conf[0][0] += 1
    if(y == 0 and x[20] == 1):
        conf[1][0] += 1
    if(y == 1 and x[20] == 0):
        conf[0][1] += 1
accuracy = correct / len(test)
precision = conf[0][0]/(conf[0][0] + conf[0][1])
recall = conf[0][0]/(conf[0][0] + conf[1][0])
f_measure = 2*precision*recall / (precision+recall)
print("F-measure:", f_measure)
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:", recall)